In [ ]:
#this notebook establishes parameter sets that will be used by LASAM

In [ ]:
#!pip install smt

In [ ]:
###note: this should be moved into the main LGAR-C folder (rather than LGAR-C/tests) to be run as it is

In [ ]:
import numpy as np
import random 
from datetime import datetime #used to record total runtime
# import os #needed to run system commands from python
import subprocess #needed to run system commands from python, more flexible and safer than os
from smt.sampling_methods import LHS #Latin hypercube sampling
import sys
import matplotlib.pyplot as plt
import re

In [ ]:
theta_r_range = [0.01 ,0.15]
theta_e_range = [0.3, 0.8]
log_alpha_range =   [-3.0, -0.52] #corresponding to 0.001 and 0.3, via log_10
n_range =       [1.01, 3] #I recommend n to be no smaller than 1.01. Weird things happen with the water retention curve below that  
log_Ks_range =      [-3, 2] #corresponding to 0.001 and 100, via log_10
initial_psi_range = [10, 3000]
with_ponded_head = [-1, 1] #later, ponded head will be set to zero or nonzero based on this
field_capacity_psi_range = [10.3, 516.5]


In [ ]:
#establishes limits for Latin hypercube.
#in this case there's going to be a new hypercube for each of the three soil layers.

xlimits = np.array([[theta_r_range[0], theta_r_range[1]],
                    [theta_e_range[0], theta_e_range[1]],
                    [log_alpha_range[0], log_alpha_range[1]],
                    [n_range[0], n_range[1]],
                    [log_Ks_range[0], log_Ks_range[1]],
                    [initial_psi_range[0], initial_psi_range[1]],
                    [with_ponded_head[0], with_ponded_head[1]],
                    [field_capacity_psi_range[0], field_capacity_psi_range[1]],
                    ])


In [ ]:
samplingx = LHS(xlimits=xlimits,random_state=3) #random_state is for reproducability
samplingy = LHS(xlimits=xlimits,random_state=4) #random_state is for reproducability
samplingz = LHS(xlimits=xlimits,random_state=5) #random_state is for reproducability

#this creates the Latin hypercube containing all parameter sets, where num is the number of parameter sets desired.
num = 10000
x = samplingx(num)
y = samplingy(num)
z = samplingz(num)

In [ ]:
x

In [ ]:
y

In [ ]:
z

In [ ]:
num_runs = 0
num_successful_runs = 0
unsuccessful_parameter_sets = []
#error types are: mass balance, timeout due to infinite loop, and nonzero ponded head when its max is set to 0.
unsuccessful_parameter_sets_ponded_water = []
unsuccessful_parameter_sets_timeout = []
unsuccessful_parameter_sets_mass_bal = []
runtime_vec = []
global_balance_vec = []

begin_time = datetime.now() #used to record total runtime

# test_set = [8400]
# for parameter_set in test_set:
# for parameter_set in range(test_set[0],num):

for parameter_set in range(num):

    x[parameter_set,2] = 10**x[parameter_set,2]
    x[parameter_set,4] = 10**x[parameter_set,4]
    y[parameter_set,2] = 10**y[parameter_set,2]
    y[parameter_set,4] = 10**y[parameter_set,4]
    z[parameter_set,2] = 10**z[parameter_set,2]
    z[parameter_set,4] = 10**z[parameter_set,4]
    print("parameter set:")
    print(parameter_set)
#     print("actual alpha:")
#     print(x[parameter_set,2])
#     print("actual Ks:")
#     print(x[parameter_set,4])
    with open('data/vG_default_params_LHS_Koptis_Farms_3_layer.dat', 'r') as file:
    # read a list of lines into data
        parameters_file_text = file.readlines()
        file.close()
#     layer_to_replace = random.randint(1, 3)
    layers_to_replace = [1,2,3]
#     print("layer_to_replace: \n")
#     print(layer_to_replace)
    for layer_to_replace in layers_to_replace:
        if layer_to_replace == 1:
            parameters_file_text[layer_to_replace] = '"Clay"\t                ' + str(x[parameter_set,0]) + '\t' + str(x[parameter_set,1]) + '\t' + str(x[parameter_set,2]) + '\t' + str(x[parameter_set,3]) + '\t' + str(x[parameter_set,4]) + '\n'    
        if layer_to_replace == 2:
            parameters_file_text[layer_to_replace] = '"Clay"\t                ' + str(y[parameter_set,0]) + '\t' + str(y[parameter_set,1]) + '\t' + str(y[parameter_set,2]) + '\t' + str(y[parameter_set,3]) + '\t' + str(y[parameter_set,4]) + '\n' 
        if layer_to_replace == 3:
            parameters_file_text[layer_to_replace] = '"Clay"\t                ' + str(z[parameter_set,0]) + '\t' + str(z[parameter_set,1]) + '\t' + str(z[parameter_set,2]) + '\t' + str(z[parameter_set,3]) + '\t' + str(z[parameter_set,4]) + '\n' 
            
    with open('data/vG_default_params_LHS_Koptis_Farms_3_layer.dat', 'w') as file:
        file.writelines( parameters_file_text )
        file.close()
        
    with open('configs/config_lasam_LHS_Koptis_Farms_3_layer.txt', 'r') as file:
    # read a list of lines into data
        config_file_text = file.readlines()
        file.close()
    config_file_text[4] = 'initial_psi=' + str(x[parameter_set,5]) + '[cm]\n'
    
    if (x[parameter_set,6]<0):
        config_file_text[8] = 'ponded_depth_max=0[cm]\n'
    else:
        config_file_text[8] = 'ponded_depth_max=2[cm]\n'
    if (x[parameter_set,6]>0.5):
        config_file_text[8] = 'ponded_depth_max=5[cm]\n'
        
    config_file_text[13] = 'field_capacity_psi=' + str(x[parameter_set,7]) + '[cm]\n'
        
    with open('configs/config_lasam_LHS_Koptis_Farms_3_layer.txt', 'w') as file:
        file.writelines( config_file_text )
        file.close()
        
    print("running LASAM ...")
#     result = !./build/lasam_standalone configs/config_lasam_LHS_Phillipsburg.txt
#     result = !./build/lasam_standalone configs/config_lasam_LHS_Koptis_Farms.txt 
    start_time_loc = datetime.now() #used to record local runtime
    result = !./build/lasam_standalone configs/config_lasam_LHS_Koptis_Farms_3_layer.txt 
    end_time_loc = datetime.now() #used to record local runtime
    runtime_loc = end_time_loc - start_time_loc
    runtime_vec.append(runtime_loc.total_seconds())
    
    for row in result:
        print(row)
    print(" ")
    print(" ")
    
    run_success = False
    surface_ponded_water = 0.0;
    if (len(result) > 1):
#         if (result[8][0:20]=='Surface ponded water'):
#             surface_ponded_water = float(result[8][30:38])
#         if ((result[2] == '-------------------- Simulation Summary ----------------- ') & (result[8]=='Surface ponded water      =   0.0000000000 cm')):
        if ('-------------------- Simulation Summary ----------------- ' in result):
            num_successful_runs = num_successful_runs + 1
            run_success = True

    if (run_success == False):
        unsuccessful_parameter_sets.append(parameter_set)
        print("unsuccessful parameter set: #################################################################################### ")
        print(parameter_set)
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        for line in parameters_file_text:
            print(line)
        print("adding code that causes the notebook to crash when an individual run fails, for testing purposes")
        sys.exit()
        if ('mass balance closure not possible within 100000 iterations. Timeout ' in result):
            unsuccessful_parameter_sets_timeout.append(parameter_set)
        if (len(result)>1):
            if (result[1]=='Local mass balance at this timestep... '):
                unsuccessful_parameter_sets_mass_bal.append(parameter_set)
                
    if (run_success):

        # Initialize the variable for the global balance number
        global_balance = None

        # Loop through the list to find the string containing "Global balance"
        for line in result:
            if "Global balance" in line:
                # Extract the number using a regular expression
                number_str = re.search(r"[-+]?\d*\.\d+e[-+]?\d+", line).group()
                # Convert the extracted string to a float
                global_balance = float(number_str)
                break  # Stop the loop once the number is found

#         print(global_balance)
        global_balance_vec.append(np.log10(abs(global_balance)))
            
            
        
    num_runs = num_runs + 1
    
end_time = datetime.now() #used to record total runtime
total_time = end_time - begin_time
print("all runs complete.")



    

In [ ]:
result

In [ ]:
print("elapsed time: ")
print(total_time)
print("num_successful_runs:")
print(num_successful_runs)
print("num_runs:")
print(num_runs)
print("success rate (%): ")
print(100*num_successful_runs/num_runs)
print("unsuccessful_parameter_sets:")
print(unsuccessful_parameter_sets)
print("unsuccessful parameter sets due to erroneous ponded water: ")
print(unsuccessful_parameter_sets_ponded_water)
print("unsuccessful parameter sets due to mass balance error: ")
print(unsuccessful_parameter_sets_mass_bal)
print("unsuccessful parameter sets due to timeout (infinite loop likely): ")
print(unsuccessful_parameter_sets_timeout)

In [ ]:
global_balance_vec_filtered = []
for item in global_balance_vec:
    if (not np.isinf(np.array(item))):
        global_balance_vec_filtered.append(item)
global_balance_vec = global_balance_vec_filtered

In [ ]:
plt.hist(global_balance_vec)

In [ ]:
plt.hist(runtime_vec)

In [ ]:
np.mean(runtime_vec)

In [ ]:
unsuccessful_parameter_sets == unsuccessful_parameter_sets_timeout

In [ ]:
# unsuccessful_parameter_sets

In [ ]:
x[unsuccessful_parameter_sets]

In [ ]:
# result[1]

In [ ]:
# unsuccessful_parameter_sets_ponded_water 

In [ ]:
# unsuccessful_parameter_sets_timeout 

In [ ]:
# unsuccessful_parameter_sets_mass_bal 

In [ ]:
# result[8][0:20]=='Surface ponded water'

In [ ]:
# result

In [ ]:
# !pwd

In [ ]:
plt.hist(global_balance_vec)

In [ ]:
count = 0
for item in global_balance_vec:
    if item>-4:
        count = count + 1
count/len(global_balance_vec)

In [ ]:
###example of how the psi-theta relationship (soil water retention curve) loses its 1:1
###behavior for small psi in the van Genuchten model
###also true for extremely large psi 

# Defining the variables
theta_r = 0.02
theta_s = 0.6
alpha = 0.001
n = 2.8
psi = 0.001

# Calculating the expression
theta = theta_r + (theta_s - theta_r) / (1 + (alpha * psi)**n)**(1 - 1/n)
theta

#note that for psi = 0.001, theta evaluates to exactly theta_s and not slightly less.

In [ ]:
# Defining the variables
theta_r = 0.1
theta_s = 0.4
alpha = 0.1
n = 1.4
psi = 0.001

# Calculating the expression
theta = theta_r + (theta_s - theta_r) / (1 + (alpha * psi)**n)**(1 - 1/n)
theta

###in contrast, note that in this case, the same psi value yields a theta that is less than
###theta_s. While the difference is small, it's enough to cause an infinite WF depth because
##when a WF crosses layer boundaries from the second soil to the first, there will be no delta
###psi that allows for mass balance closure because theta will not change wrt psi 